In [1]:
import nibabel as nib
import numpy as np
from neuromaps import datasets, images


def cifti_to_surface(data, axis, surf_name):
    """select specified surface from CIFTI object"""
    for name, idx, model in axis.iter_structures():  # volumetric & surface
        if name == surf_name:
            data = data.T[idx]
            vertex_idx = model.vertex  # 1-N, except medial wall
            surf_data = np.zeros((vertex_idx.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vertex_idx] = data
            return surf_data

    raise ValueError(f"no structure named {surf_name}")


annotation = datasets.fetch_annotation(
    source="hcps1200", desc="myelinmap", space="fsLR", den="32k"
)

dlabel = "/Volumes/griegner/hcp/HCP_PTN1200_recon2/groupICA/groupICA_3T_HCP1200_MSMAll_d300.ica/melodic_IC_ftb.dlabel.nii"
gifti = images.dlabel_to_gifti(dlabel)  # cifti to surface: removed volumetric data

# test if dlabel_to_gifti() is removing volumetric *only* volumetric data
cifti = nib.load(dlabel)
annotation = cifti.get_fdata(dtype=np.float32)
axis = cifti.header.get_axis(1)
surface_l = cifti_to_surface(annotation, axis, "CIFTI_STRUCTURE_CORTEX_LEFT")
surface_r = cifti_to_surface(annotation, axis, "CIFTI_STRUCTURE_CORTEX_RIGHT")

assert np.array_equal(surface_l.flatten(), gifti[0].agg_data()), "cortex left"
assert np.array_equal(surface_r.flatten(), gifti[1].agg_data()), "cortex right"
